In [1]:
import numpy as np
import torch 
import torch.nn as nn
from tqdm import tnrange
import torch.optim as optim
import torch.nn.functional as F
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score
import scipy
import time

from models.sub_cos_glm import Sub_Cos_GLM
#from models.sub_tcn import Sub_TCN
from models.gru import GRU
from models.gru_stacked import GRU_Stacked
from models.sub_cos_glm_stacked import Sub_Cos_GLM_Stacked
from models.gru_multilayer import GRU_Multilayer
from models.sub_cos_glm_multilayer import Sub_Cos_GLM_Multilayer

In [2]:
base_dir = "/media/hdd01/sklee/"
experiment = "clust12-20_noNA"
cell_type = "CA1"
E_neural_file = "Espikes_neural.npz"
I_neural_file = "Ispikes_neural.npz"
clust_mode = "hand"
model_type = "glm"

E_neural = scipy.sparse.load_npz(base_dir+cell_type+"_"+experiment+"/data/"+E_neural_file)
I_neural = scipy.sparse.load_npz(base_dir+cell_type+"_"+experiment+"/data/"+I_neural_file)

if (clust_mode == "hand") or (clust_mode == "whole") or (clust_mode == "global"):
    C_syn_e = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/handsub18_C_syn_e.npy")
    C_syn_i = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/handsub18_C_syn_i.npy")
    C_syn_e = torch.from_numpy(C_syn_e).float()
    C_syn_i = torch.from_numpy(C_syn_i).float()
elif clust_mode == "rand":
    C_syn_e = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/randsub5_C_syn_e.npy")
    C_syn_i = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/randsub5_C_syn_i.npy")
    C_syn_e = torch.from_numpy(C_syn_e).float()
    C_syn_i = torch.from_numpy(C_syn_i).float()

#C_syn_e = np.load("/media/hdd01/sklee/CA1_clust8-30/data/handsub1_C_syn_e.npy")
#C_syn_i = np.load("/media/hdd01/sklee/CA1_clust8-30/data/handsub1_C_syn_i.npy")
#C_syn_e = torch.from_numpy(C_syn_e).float()
#C_syn_i = torch.from_numpy(C_syn_i).float()

In [3]:
H_no = 2
sub_no = 18
E_no = 2000
I_no = 200
T_no = 500
device = torch.device("cpu")

In [4]:
if model_type == "gru":
    model = GRU(C_syn_e.to(device), C_syn_i.to(device), H_no, device)
elif model_type == "grustack":
    model = GRU_Stacked(C_syn_e.to(device), C_syn_i.to(device), H_no, device)
elif model_type == "glm":
    model = Sub_Cos_GLM(C_syn_e.to(device), C_syn_i.to(device), T_no, H_no, device)
elif model_type == "glmstack":
    model = Sub_Cos_GLM_Stacked(C_syn_e.to(device), C_syn_i.to(device), T_no, H_no, device)
elif model_type == "grumulti":
    model = GRU_Multilayer(C_syn_e.to(device), C_syn_i.to(device), H_no, device)
elif model_type == "glmmulti":
    model = Sub_Cos_GLM_Multilayer(C_syn_e.to(device), C_syn_i.to(device), T_no, H_no, device)
    
model.to(device).float()
model.load_state_dict(torch.load(base_dir+cell_type+"_"+experiment+"/"+clust_mode+"/"+model_type+"_s"+str(sub_no)+"_h"+str(H_no)+".pt", map_location='cpu'))
#model.load_state_dict(torch.load(base_dir+cell_type+"_"+experiment+"/"+clust_mode+"/"+model_type+"_s"+str(sub_no_file)+"_h"+str(H_no)+"_set5.pt", map_location='cuda:0'))
model.eval()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

4433


In [5]:
test = np.zeros((1000,50000))
batch_size = 5

for i in tnrange(1000//batch_size):
    part_E_neural = torch.from_numpy(E_neural[i*batch_size*50000:(i+1)*batch_size*50000].toarray().reshape(batch_size,50000,-1)).to(device).float()
    part_I_neural = torch.from_numpy(I_neural[i*batch_size*50000:(i+1)*batch_size*50000].toarray().reshape(batch_size,50000,-1)).to(device).float()
    #part_test, _ = model(part_E_neural, part_I_neural)
    part_test, _, _ = model(part_E_neural, part_I_neural)
    test[i*batch_size:(i+1)*batch_size] = part_test.cpu().detach().numpy().reshape(batch_size,50000)
        
np.save(base_dir+cell_type+"_"+experiment+"/"+clust_mode+"/"+model_type+"_s"+str(sub_no)+"_h"+str(H_no)+"_Vall.npy", test)


/tmp/ipykernel_17894/4052821804.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(1000//batch_size):


  0%|          | 0/200 [00:00<?, ?it/s]